# Imports

In [2]:
import time
from langdetect import detect
import langid

In [3]:
import asyncio
import aiohttp
from aiohttp import ClientSession, ClientTimeout

In [4]:
import json
import pickle
import numpy as np
import pandas as pd
import re

/tmp/ipykernel_7193/3146907529.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [5]:
import requests
from bs4 import BeautifulSoup
import html

In [6]:
import os

In [7]:
import cProfile

In [ ]:
pkl_data = pd.read_pickle('datas/bestiary_ch_su.xz')

# Parse from bestiary.su

## One page


In [12]:
response = requests.get('https://www.bestiary.us/contents')
response.text

'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">\n <html xmlns="http://www.w3.org/1999/xhtml" lang="ru" xml:lang="ru">\n<head>\n  <title>Алфавитный указатель мифических существ | Bestiary.us</title>\n  <meta http-equiv="Content-Type" content="text/html; charset=utf-8" />\n<link rel="shortcut icon" href="/files/favicon.ico" type="image/x-icon" />\n  <style type="text/css" media="all">@import "/sites/all/modules/fivestar/css/fivestar.css";</style>\n<style type="text/css" media="all">@import "/files/fivestar/basic/basic.css";</style>\n<style type="text/css" media="all">@import "/sites/all/modules/nodereference_views_select/nodereference_views_select.css";</style>\n<style type="text/css" media="all">@import "/modules/book/book.css";</style>\n<style type="text/css" media="all">@import "/modules/extra_voting_forms/widgets/default/extra_voting_forms_all.css";</style>\n<style type="text/css" media="all">@import "/modules/img_assist/

In [43]:
soup = BeautifulSoup(response.text, "html.parser")
# idx = 267
href = []
href_cut = soup.find('div', {'id':'block-indexs'})
only_rus = ('').join(str(href_cut).split('<a href="#home">к началу</a>')[1:31])
soup = BeautifulSoup(only_rus, "html.parser")
href_tags = soup.find_all('span', {'class':'block-index-item-title'})
for hr in href_tags:
    href.append('https://www.bestiary.us' + str(hr.find('a').get('href')))

In [44]:
href[-1], len(href)

('https://www.bestiary.us/yajuj', 4758)

In [17]:
response_test = requests.get('https://www.bestiary.us/bao-ku')
response_test.text

'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">\n <html xmlns="http://www.w3.org/1999/xhtml" lang="ru" xml:lang="ru">\n<head>\n  <title>А Бао Ку | Bestiary.us</title>\n  <meta http-equiv="Content-Type" content="text/html; charset=utf-8" />\n<link rel="next" href="/blogs/alke/bao-qou-statya-iz-entsiklopedii-vampirskoi-mifologii-t-bein" />\n\n<link rel="alternate" hreflang="x-default" href="//www.bestiary.us/abaoku" />\n<link rel="alternate" hreflang="ru" href="//www.bestiary.us/abaoku/ru" />\n<link rel="alternate" hreflang="en" href="//www.bestiary.us/abaoku/en" />\n<meta property="og:url" content="//www.bestiary.us/abaoku" />\n<meta property="og:title" content="А Бао Ку — некая описанная Х.Л.Борхесом в его &quot;Книге вымышленных существ&quot; мифическая сущность, чувствительная ко всем оттенкам человеческой души" />\n<meta property="og:type" content="article" />\n<meta property="og:description" content="Некая описанная Х.Л

In [ ]:
# Подрезать всё до тайтла
# Обрезать всё после описания
# Взять все абзацы
# Убрать все кроме русского

In [65]:
time_s = time.time()
soup_test_content = BeautifulSoup(response_test.text, 'html.parser')
for_det = soup_test_content.find_all('div', {'itemprop':"content", 'lang':'ru'})[0]
time_e = time.time()
print(time_e - time_s)

0.05585217475891113


In [153]:
s_t = time.time()
soup_test_title = BeautifulSoup(response_test.text, 'html.parser')
soup_test_title.find('span', {'itemprop':"title"}).text
e_t = time.time()
print(e_t - s_t)

0.0621490478515625


In [102]:
soup = BeautifulSoup(str(for_det), 'html.parser')
soup.find_all('p')

[<p></p>,
 <p>Хорхе Луис Борхес в своей "Книге вымышленных существ" пишет:</p>,
 <p class="ep">Para contemplar el paisaje más maravilloso del mundo, hay que llegar al último piso de la Torre de la Victoria, en Chitor. Hay ahí una terraza circular que permite dominar todo el horizonte. Una escalera de caracol lleva a la terraza, pero sólo se atreven a subir los no creyentes de la fábula, que dice así:</p>,
 <p class="ep">"En la escalera de la Torre de la Victoria, habita desde el principio del tiempo el <a class="self_link">A Bao A Qu</a>, sensible a los valores de las almas humanas. Vive en estado letárgico, en el primer escalón, y sólo goza de vida consciente cuando alguien sube la escalera. La vibración de la persona que se acerca le infunde vida, y una luz interior se insinúa en él. Al mismo tiempo, su cuerpo y su piel casi translúcida empiezan a moverse. Cuando alguien asciende la escalera, el <a class="self_link">A Bao A Qu</a> se coloca casi en los talones del visitante y sube pr

In [101]:
s_t = time.time()
for text in for_det:
    if len(text)>2:
        if detect(text.text) == 'ru':
            print(text.text)
e_t = time.time()
print(e_t - s_t)

## Он как-то криво работает

А Бао А Ку — один из самых спорных персонажей "Книги вымышленных существ" Хорхе Луиса Борхеса. Великий испанский мистификатор в разных изданиях "Книги вымышленных существ" то говорит нам, что эту легенду приводит С.С.Итурвуру в приложении к его ставшему классическим трактату "О малайском волшебстве" (1937) (1: с.6), то ссылается на капитана сэра Ричарда Фрэнсиса Бёртона, указывая, что тот упоминал А Бао Ку в предисловии к своему переводу сказок "Тысячи и одной ночи". Но так или иначе, слишком велика вероятность, что ни упоминания легенды, ни трактата, ни даже некоего Итурвуру — далее чем в воображении самого Борхеса вовсе не существовало.
Как оказалось, многие старики местных деревень слышали в молодости историю о неком Абанг Аку. В ней говорилось о спиральной лестнице, соединяющей одно измерение с другим. Лестница, прежде используемая звездными богами для перемещений и перевоплощений, становилась для некоторых из них опасной ловушкой времени, не пускающей их в их собственную вселенную

In [90]:
monster_data = pd.DataFrame(columns=['name', 'description'])

## Other pages


In [ ]:
'''
Функция делает следующее:
1. Берет страницу
2. Обрезает "лишнее"
3. Ищет название, кладет в колонку дф name
4. Парсит описание, кладет в колонку дф description
'''

In [ ]:
monster_data = pd.DataFrame(columns=['name', 'description'])

In [56]:
def do_monster(url, text_list=None):
    if text_list==None:
        text_list = []
    
    response = requests.get(url)
    #create soup
    soup_text = BeautifulSoup(response.text, 'html.parser')
    
    #add descr and title to data
    title = soup_text.find('h1', {'class':"title"}).text
    page_center = soup_text.find_all('div', {'itemprop':"content", 'lang':'ru'})
    
    soup = BeautifulSoup(str(page_center), 'html.parser')
    descr = soup.find_all('p')
    
    for text in descr:
        t = text.text
        if len(t)>2:
            if detect(t) == 'ru':
                text_list.append(t)
    
    descr = ' '.join(text_list)
#     print({data.columns[0]:title, data.columns[1]:descr})
    new_data = pd.DataFrame({'name':[title], 'description':[descr]})
    
    return new_data

In [ ]:
with cProfile.Profile() as pr:
    monster_data = do_monster('https://www.bestiary.us/bao-ku')

    pr.print_stats()

In [131]:
data_list = []
start_time = time.time()
timeout = ClientTimeout(total=600)
errors_list = []

async def get_page_data(session, url):

    async with session.get(data=monster_data, url=url, timeout=timeout) as response:
        if response.status != 200:
            errors_list.append(url)

        else:
            response_text = await response.text()
            
            text_list = []
            soup = BeautifulSoup(response_text, 'html.parser')

            title_try = soup.find(
                'h1', {'class':"title"})
            
            try:
                title = title_try.text

            except AttributeError:
                print(title_try)
                print(url)
                print(response)
                raise
            
            page_center = soup.find_all(
                'div', {'itemprop':"content", 'lang':'ru'})
            
            soup_c = BeautifulSoup(str(page_center), 'html.parser')
            if soup_c:
                descr = soup_c.find_all('p')
                for text in descr:
                    try:
                        t = text.text
                        if len(t)>2:
                            if detect(t) == 'ru':
                                text_list.append(t)
                    except: 
                        print(url)
                        print('No features in text.')
                        print(text.text)
                        if len(text.text) > 2:
                            errors_list.append(url)

                descr = ' '.join(text_list)

            else: 
                descr = url
                print(url)
                raise

                
            new_data = pd.DataFrame({'name':[title], 'description':[descr]})

            print(f'Обработал страницу {title}')
            data_list.append(new_data)
    
async def gather_data(href_tags):

    async with ClientSession() as session:
        step = 20
        
        for i in range(len(href_tags) // step):
            tasks = []
            for page in href_tags[i*step:(i+1)*step]:
                task = asyncio.create_task(get_page_data(session, page))
                tasks.append(task)
        
            # loop = asyncio.get_event_loop()
            # loop.run_until_complete(asyncio.gather(*tasks))
            await asyncio.gather(*tasks)
            print(f'Закончил {i} шаг из {len(href_tags) // step}')
    
    finish_time = time.time() - start_time
    print(f'Затраченное на работу скрипта время: {finish_time}')

In [132]:
await gather_data(href)

Обработал страницу Абалжаргана
Обработал страницу Абизу
Обработал страницу Абнауаю
Обработал страницу Абайа
Обработал страницу А Бао Ку
Обработал страницу Ааче
Обработал страницу А-Ми-Кук
Обработал страницу А Бао Ку
Обработал страницу Абада
Обработал страницу А Бао Ку
Обработал страницу Абак
Обработал страницу Абайа
Обработал страницу Аврага Могой
Обработал страницу Абасы
Обработал страницу Абасы
Обработал страницу Абраксас
Обработал страницу Абатва
Обработал страницу Абасы
Обработал страницу Абат
Обработал страницу Абасы
Закончил 0 шаг из 237
Обработал страницу Абчанчу
Обработал страницу Агач киши
Обработал страницу Агогве
Обработал страницу Аврага Могой
Обработал страницу Аврага Могой
Обработал страницу Абизу
Обработал страницу Абура-сумаси
Обработал страницу Агишки
Обработал страницу Абура-акаго
Обработал страницу Аванк
Обработал страницу Авгур
Обработал страницу Седна
Обработал страницу Агишки
Обработал страницу Абуми-гути
Обработал страницу Огневик
Обработал страницу Абуми-гути
Об

In [135]:
monster_data = pd.concat(data_list, ignore_index=True)
monster_data.shape

(4349, 2)

In [176]:
monster_data = monster_data.drop_duplicates(subset=['name'])

# Work wiith dataframe

In [8]:
#monster_data.to_pickle('monster_data.xz')
# monster_data = pd.read_pickle('datas/monster_data.xz')

In [ ]:
'''
- [x] Убрать белорусские вставки 
- [x] Убрать указания на источники и года (с.123), (488: с.178), (1150: н.8, с.69), (1930), (350: с.173; 488: с.178), ст.39, (829: s.113), (958: p.41-44)
- [x] Поменять \n -> '. '
- [x] Убрать славянский текст "Среди горъ, въ кегорахъ (мѣстностяхъ, покрытыхъ ягелемъ, на которых пасутся олени) и мѣстахъ покрытыхъ травой живутъ Гофиттеракъ"
'''

In [11]:
monster_data = monster_data.loc[monster_data['name'] != 'Новый Бестиарий: вымышленные животные и расы, мифологические и мифические существа, фантастические чудовища и даже монстры!']
monster_data.drop_duplicates(subset=['name'], inplace = True)
monster_data.reset_index(drop=True, inplace=True)

In [12]:
monster_data

,name,description
0,Абалжаргана,Абалжаргана — зооморфный персонаж низшей мифол...
1,Абизу,Демон по имени Абизу впервые появляется в трак...
2,Абнауаю,"Абнауаю́ (буквально ""лесной человек"") — мифоло..."
3,Абайа,"В фольклоре папуасов Новой Гвинеи, Соломоновых..."
4,А Бао Ку,"Хорхе Луис Борхес в своей ""Книге вымышленных с..."
...,...,...
2049,Яогуай,Яогуай — в китайском фольклоре демоны и призра...
2050,Ямаваро,"Согласно японской мифологии, Ямаваро — низшие ..."
2051,Ямата-но Ороти,"Ямата-но Ороти — легендарный японский дракон, ..."
2052,Янари,"Янари — крохотные демоны-о́ни, что появляются ..."


In [124]:
monster_data_listed = monster_data.copy()

In [125]:
patt = re.compile(r'\([0-9\s\.сcspg:,;н-]*\)') # Убрать указания на источники и года (с.123), (488: с.178), (1150: н.8, с.69), (1930), (350: с.173; 488: с.178), (829: s.113), (958: p.41-44)
monster_data_listed.description = monster_data.description.str.replace(patt, '', regex=True)

In [126]:
monster_data_listed.loc[monster_data_listed['name'] == 'Абасы']

,name,description
10,Абасы,"Верхнего мира абаасы — \nИздревле наши враги, ..."


In [127]:
# Уберу служебные символы и пробелы длиной >2 заменю на одинарные
monster_data_listed.description = monster_data_listed.description.str.replace('\n', ' ') 
monster_data_listed.description = monster_data_listed.description.str.replace('\u200b', ' ')
monster_data_listed.description = monster_data_listed.description.str.replace('\xa0', ' ')

monster_data_listed.description = monster_data_listed.description.str.replace('\s{2,}', ' ', regex=True)
monster_data_listed.description = monster_data_listed.description.str.replace('[\s\.]{2,}', '. ', regex=True)

In [128]:
monster_data_listed.loc[monster_data_listed['name'] == 'Абасы']

,name,description
10,Абасы,"Верхнего мира абаасы — Издревле наши враги, Ни..."


In [129]:
list(monster_data_listed.loc[monster_data.description.str.contains('ѣ')].description.sample(5))

['Стухач — в сербском фольклоре демоническое существо ("дьявол" или "вила", не путать со Здухачем), делающее себе обувь из человеческих связок. Особенно известен этот персонаж в Герцеговине. По Херцеговини приповиједају да се стухаћи (као виле или ђаволи) налазе по великијем планинама и по камењацима, и имају на ногама гужве од људскијех жила, да им се ноге не клизају по страни и по камењу; кад се коме прекине гужва, а он ухвати човјека и извуче му жиле из ногу, те начини другу. Вук Караџић "Живот и обичаји народа српскога". III. Вјеровање ствари којијех нема В Герцеговине рассказывают, что стухачи (как вилы или дьяволы) живут в высоких горах и скалах, и носят на ногах обувь из человеческих жил, чтобы их ноги не скользили на камнях; когда обувь изнашивается, стухач ловит человека и вытягивает его жилы с ног, чтобы сделать новую. Вук Караджич "Жизнь и обычаи народа сербского". III. Вера в то, чего нет Въ каменистыхъ горахъ обитаетъ грозный духъ Стуачь. Чтобъ не оступиться, онъ носитъ — 

### Очистка от славянского и английского, скобок, собственного списка стоп-слов.

In [ ]:
'''
1.  Дропнуть (**) <...> и аналоги. 
2. Убрать упоминания авторов. r'[А-Я]\.[А-Я]\.[А-Я+]' или обратная ситуация
3. Разделяем по .:; (с сохранением разделителя!)
    - Если в оставшемся предложении меньше k символов - убираем его
4. Если stop_words = ['книг', 'глава', 'стр', 'сборник', 
                    'перевод', 'аниме', 'сериал', 'комикс',
                    'экран', 'фильм', 'сверхъестественн', 'словарь',
                    'сказани', 
                    'белорус', 'белару', 'республ', 'район',
                    'легенд', 'крестьян'] 
    в тексте -> убрать часть текста
5. Ищем в отделенном кусочке либо англ символ, либо что-то из славяноспецифичных. Выбрасываем кусок такого предложения. 
6. Склеиваем обратно, проверяем наличие "въ".
'''

In [130]:
# patt_authwithaster = re.compile(r'\.[А-Яа-я\s0-9,]*\s[А-Яа-я]+\s\(\*+\)') # убирает авторов со звездочкой: Васильков (*) и (**) -> '. '
patt_bracketrd = re.compile(r'\([^()]*\)') #Убирает всё, что в круглых скобках "()".
patt_bracketsq = re.compile(r'\[[^\[\]]*\]') #Убирает всё, что в квадратных скобках "[]".
patt_pass = re.compile(r'<\.{1,4}>') #Убирает <...>
regex_patt_list = [patt_bracketrd, patt_bracketsq, patt_pass]

In [131]:
stop_words = ['книг', 'глава', 'стр.', 'сборник', 
            'перевод', 'аниме', 'сериал', 'комикс',
            'экран', 'фильм', 'сверхъестественн', 'словарь', 'эпос'
            'сказани', 'манга', 'христиан', 'православ',
            'белорус', 'белару', 'республ', 'район',
            'японск', 'русск', 'второй мировой', 'вторая мировая', 'вторую мировую',  
            'крестьян', 'вселен', 'турист'] 

In [132]:
for patt in regex_patt_list:
    monster_data_listed.description = monster_data_listed.description.str.replace(patt, '', regex=True)

In [133]:
monster_data_listed.loc[monster_data_listed['name'] == 'Абасы']

,name,description
10,Абасы,"Верхнего мира абаасы — Издревле наши враги, Ни..."


In [134]:
patt_slav = re.compile(r'[A-Za-zіѣѧѫѡѩөўἄλ]')

In [135]:
text = 'Сиды - в ирландской мифологии божественные существа, живущие внутри холмов в ирландской мифологии божественные существа, живущие внутри холмов.'
num_of_repet = 3

def clear_from_repet(text, num_of_repet=4, num_of_words=4):
    text_split = text.split()
    repet_list = []
    if len(text_split) < num_of_words:
        num_of_repet = len(text_split) - 1
        num_of_words = len(text_split) - 1

    for i in range(num_of_words):
        word = text_split[i]
        for j in range(i+1, len(text_split)):
            if word == text_split[j]:
                repet_list.append(i) #Добавлю индекс, с которого нужно обрезать
                repet_list.append(j) 
                for k in range(1, num_of_repet):
                    try:
                        if k == num_of_repet-1 and text_split[i+k] == text_split[j+k]:
                            text_new = ' '.join(text_split[:repet_list[0]] + text_split[repet_list[1]:]) #Если последние слова равны, то мы режем последовательность и возвращаем её.
                            return text_new

                        if text_split[i+k] != text_split[j+k]: #Если последующие num_of_repet-1 слов также равны, продолжаем сравнивать. Иначе брейк.
                            repet_list = []
                            break
                    except:
                        print(f'k: {k}, i:{i}, j:{j}, \n {text}')
                        raise
            else:
                repet_list = [] #Если это было ложное срабатывание и повторов недостаточно, чистим лист
    return

In [48]:
clear_from_repet(text)

Очистил текст. Было: Сиды - в ирландской мифологии божественные существа, живущие внутри холмов в ирландской мифологии божественные существа, живущие внутри холмов., 
 Стало: Сиды - в ирландской мифологии божественные существа, живущие внутри холмов.


'Сиды - в ирландской мифологии божественные существа, живущие внутри холмов.'

In [136]:
# list(monster_data_listed.description.apply(lambda x: re.findall('[^.]+\.', x)).head())

def check_me(texts):
    temp_list = []
    texts = re.findall('[^.]+\.', texts)
    
    for text in texts:
        text = text.lower()
        if len(text) > 4:
            if len(text.split()) > 2:
                if langid.classify(text)[0] == 'ru':
                    if not re.search(patt_slav, text):
                        if not any(word in text for word in stop_words): 
                            if not re.search(patt_auth, text):
                                temp_list.append(text)

    if len(temp_list) == 0:
        return #возвращаем none в случае, если после очистки ничего не осталось, чтобы потом удалить строку.
    elif len(temp_list) == 1:
        clear_from_repet(temp_list[0])
    else: #Т.е. длина последовательности от двух элементов  
        text_for_c = temp_list[0] + temp_list[1] #Возможные повторы: 'aaa, bbb. aaa, bbb.' или 'aaa aaa. bbb.'
        new_text = clear_from_repet(text_for_c)
        if new_text: # Иначе просто return
            new_text = re.findall('[^.]+\.', new_text)
            if (len(new_text) > 1) & (len(temp_list) > 2):
                print(f'(if) Было: "{temp_list}", \n Стало: "{new_text[0], temp_list[2:]}"')
                temp_list = new_text[0], temp_list[2:]
                
            else: #Длина ровно два. То есть оба элемента ушли в функцию
                print(f'(Else) Было: "{temp_list}", \n Стало: "{new_text}"')
                return ''.join(new_text)
                
    return ''.join(temp_list)


In [137]:
monster_data_listed.loc[monster_data_listed['name'] == 'Абасы']

,name,description
10,Абасы,"Верхнего мира абаасы — Издревле наши враги, Ни..."


In [122]:
# monster_data_listed.description.sample(10).apply(lambda x: check_me(x))
monster_data_listed.loc[monster_data_listed['name'] == 'Абасы'].description.apply(lambda x: check_me(x))

10    верхнего мира абаасы — издревле наши враги, ни...
Name: description, dtype: object

In [138]:
monster_data_checked = monster_data_listed.copy()
monster_data_checked.description = monster_data_listed.description.apply(lambda x: check_me(x))

(Else) Было: "['злобный демонический дух в фольклоре народа мандайя с филиппинского острова минданао, а также имя собственное одного из божеств народа багобо.', ' злобный демонический дух в фольклоре народа мандайя с филиппинского острова минданао, а также имя собственное одного из божеств народа багобо.']", 
 Стало: "['злобный демонический дух в фольклоре народа мандайя с филиппинского острова минданао, а также имя собственное одного из божеств народа багобо.']"
(Else) Было: "['"в монгольских легендах гигантский дракон-змея с 27 головами и 33 хвостами, заключенный в подземной крепости".', ' "в монгольских легендах гигантский дракон-змея с 27 головами и 33 хвостами, заключенный в подземной крепости".']", 
 Стало: "['"в монгольских легендах гигантский дракон-змея с 27 головами и 33 хвостами, заключенный в подземной крепости".']"
(Else) Было: "[' считается, что слово "ада" появилось у филиппинцев благодаря испанским колонизаторам как упрощение заимствованного из испанского языка слова "э

In [22]:
monster_data_listed.description.sample(1).item()

'мифология южной африки, как и многих других регионов, богата различными водяными существами, похожими на европейских русалок. в фольклоре зимбабве водяных дев называют мондао. это бледнокожие иглозубые чудовища, которые живут в реках и не прочь утащить под воду, на верную смерть, рыбаков и пловцов. и хотя многие видят в таких предания просто жуткие сказки, есть немало зимбабвийцев, которые верят, что они действительно существуют, доказательством чему — ряд инцидентов, произошедших уже в наши дни. так, в 2012 году работы, проводимые на плотинах гокве в мидленде и осборна в маникаленде, на водохранилищах вблизи городов гокве, маникаленд и мутаре, были приостановлены, потому что рабочие отказались выходить на смену, утверждая, что их терроризировали скрывающиеся там русалки, которые, с их слов, выглядели как бледнокожие люди с черными волосами и рыбьими хвостами. нкомо сказал, что белых рабочих пригласили, потому что они не были столь суеверны, но даже они утверждали, что заметили сущест

In [23]:
monster_data_checked.to_pickle('datas/bestiary_ch_su.xz')

## Playground

In [79]:
import langid

In [81]:
langid.classify("рухавы скача чалавечак")[0]

'be'

In [25]:
%pip install -U spacy>=3.0

Note: you may need to restart the kernel to use updated packages.


In [27]:
import spacy

In [28]:
nlp = spacy.load('ru_core_news_md')

In [29]:
sent = 'Перечисляя духов кладов он пишет: "В Белоруссии этот дух кладов превратился в маленького бога — Копшу, которого просят указать место кладов и помочь их отрыть, а при удаче благодарят, оставляя в его пользу известную часть добычи. славящих Бога, и аццко-демонических прислужников Дьявола. Белова О.В. "Славянский бестиарий" Книга 10, 31-32  P.S. А еще есть люди-аисты.'
sent = re.sub('[^\wА-Яа-я\s]', ' ', sent)
sent = re.sub('\s{2,}', ' ', sent)
sent = sent.lower()

In [ ]:
i = ['a', 'b', 'c', 'v', 'g', 'd', 'f']
k = i[:1]+ i[3:]
k = [i[0], i[1]]
k

['a', 'b']

In [30]:
document = nlp(sent)
for token in document:
    print(token, token.pos_) ## Вероятно, стоит всё же оставить propn, иначе потеряется прям много текста в кусках, где упоминаются места. "В белоруссии" условно??

перечисляя VERB
духов NOUN
кладов NOUN
он PRON
пишет VERB
в ADP
белоруссии PROPN
этот DET
дух NOUN
кладов NOUN
превратился VERB
в ADP
маленького ADJ
бога NOUN
копшу NOUN
которого PRON
просят VERB
указать VERB
место NOUN
кладов NOUN
и CCONJ
помочь VERB
их PRON
отрыть VERB
а CCONJ
при ADP
удаче NOUN
благодарят VERB
оставляя VERB
в ADP
его DET
пользу NOUN
известную ADJ
часть NOUN
добычи NOUN
славящих VERB
бога NOUN
и CCONJ
аццко NOUN
демонических ADJ
прислужников NOUN
дьявола NOUN
белова PROPN
о ADP
в ADP
славянский ADJ
бестиарий NOUN
книга NOUN
10 NUM
31 NUM
32 NUM
p X
s X
а CCONJ
еще ADV
есть VERB
люди NOUN
аисты NOUN
